<a href="https://colab.research.google.com/github/OneFineStarstuff/OneFineStarstuff/blob/main/Adversarial_Training_for_Enhanced_Security.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Define the model (example: simple feedforward neural network)
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Generate adversarial examples using FGSM (Fast Gradient Sign Method)
def generate_adversarial_example(model, data, target, epsilon):
    data.requires_grad = True
    output = model(data)
    loss = nn.CrossEntropyLoss()(output, target)
    model.zero_grad()
    loss.backward()
    data_grad = data.grad.data
    adv_data = data + epsilon * data_grad.sign()
    adv_data = torch.clamp(adv_data, 0, 1)  # Keep data within valid range
    return adv_data

# Define the training process
def adversarial_training(model, data_loader, optimizer, epsilon, device):
    model.train()
    for data, target in data_loader:
        data, target = data.to(device), target.to(device)

        # Generate adversarial examples
        adv_data = generate_adversarial_example(model, data, target, epsilon)

        # Training on adversarial examples
        optimizer.zero_grad()
        output = model(adv_data)
        loss = nn.CrossEntropyLoss()(output, target)
        loss.backward()
        optimizer.step()

# Main script to set up model, data, and training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MyModel().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Example data (replace with your actual dataset)
X_train = torch.randn(1000, 784)  # Example data
y_train = torch.randint(0, 10, (1000,))  # Example labels
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Set the adversarial attack strength
epsilon = 0.1

# Train the model with adversarial examples
adversarial_training(model, train_loader, optimizer, epsilon, device)